In [2]:
import ibapi
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
import threading
import time

In [4]:
class IBapi(EWrapper, EClient):
    def __init__(self):
        EWrapper.__init__(self)
        EClient.__init__(self, wrapper=self)

class GridTradingBot(IBapi):
    def __init__(self, symbol, grid_size, quantity):
        super().__init__()
        self.symbol = symbol
        self.grid_size = grid_size
        self.quantity = quantity
        self.positions = []
        self.current_price = None  # Initialize current_price

    def start(self):
        self.connect("127.0.0.1", 7497, 1)
        thread = threading.Thread(target=self.run)
        thread.start()
        time.sleep(1)  # Sleep interval to allow time for connection to server

    def nextValidId(self, orderId: int):
        self.nextOrderId = orderId
        self.place_initial_orders()

    def place_initial_orders(self):
        # Place initial buy and sell orders
        current_price = self.get_current_price(self.symbol)
        if current_price is not None:
            buy_price = current_price - self.grid_size
            sell_price = current_price + self.grid_size
            self.place_order(self.symbol, "BUY", self.quantity, buy_price)
            self.place_order(self.symbol, "SELL", self.quantity, sell_price)

    def place_order(self, symbol, action, quantity, price):
        order = ibapi.order.Order()
        order.action = action
        order.totalQuantity = quantity
        order.orderType = "LMT"
        order.lmtPrice = price
        self.placeOrder(self.nextOrderId, self.create_contract(symbol), order)
        self.nextOrderId += 1

    def create_contract(self, symbol):
        contract = ibapi.contract.Contract()
        contract.symbol = symbol
        contract.secType = "STK"
        contract.exchange = "SMART"
        contract.currency = "USD"
        return contract

    def get_current_price(self, symbol):
        # Implement a method to get the current price of the symbol
        # This is a placeholder implementation
        # Replace with actual price fetching logic
        contract = self.create_contract(symbol)
        self.reqMktData(1, contract, "", False, False, [])
        time.sleep(2)  # Sleep to allow time for price to be fetched
        print(f"Current price for {symbol}: {self.current_price}")
        return self.current_price

    def tickPrice(self, reqId, tickType, price, attrib):
        if tickType == 4:  # Last price
            self.current_price = price

    def orderStatus(self, orderId, status, filled, remaining, avgFillPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
        print(f"OrderStatus. Id: {orderId}, Status: {status}, Filled: {filled}, Remaining: {remaining}, LastFillPrice: {lastFillPrice}")

    def execDetails(self, reqId, contract, execution):
        print(f"ExecDetails. ReqId: {reqId}, Symbol: {contract.symbol}, SecType: {contract.secType}, Currency: {contract.currency}, ExecId: {execution.execId}, OrderId: {execution.orderId}, Shares: {execution.shares}, LastLiquidity: {execution.lastLiquidity}")

    # Example usage
    bot = GridTradingBot("MBT", 1, 0.5)
    bot.start()


NameError: name 'GridTradingBot' is not defined